In [75]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from collections import Counter
from numpy import mean
from numpy import std
from imblearn.over_sampling import SMOTE
from sklearn.externals import joblib
import json

In [76]:
path = '/Users/pedrojunqueira/Desktop/explore_australia/MyNBs/tile_features'

In [77]:
train_feature_paths = [path for path in os.listdir(path) if 'test' not in path]


In [78]:
df_template = pd.read_csv('./tile_features/filtered_thorium_ppm.csv')
df_template = df_template.set_index('tile_id')
df_template = df_template.drop(columns='mean_')
df_template

for file in train_feature_paths:
    if file == 'stamps_label.csv':
        continue
    else:

        df = pd.read_csv(f'./tile_features/{file}')
        col_name = file.split('.')[:-1][0]
        df.rename(columns={'mean_':col_name},inplace=True)
        df = df.set_index('tile_id')
        df_template = df_template.join(df)

    


In [113]:
df_template

,filtered_thorium_ppm,aloh_groun_content,thermal_infrared_gypsum_index,mgoh_group_content,variable_reduction_to_pole,filtered_terrestrial_dose,tir_quartz_index,ferrous_iron_index,total_magnetic_intensity,opaque_index,bouger_gravity_anomaly,kaolin_group_index,ferric_oxide_content,filtered_uranium_ppm,filtered_potassium_pct,thermal_infrared_silica_index,ferrous_iron_content,isostatic_residual_gravity_anomaly,aloh_group_composition
tile_id,,,,,,,,,,,,,,,,,,,
82771002_1,8.805973,1.207136,2.002400,1.346882e-03,24.531391,39.618580,0.499420,0.005771,226.028381,0.254697,-490.953339,1.367060e-03,0.665925,1.517591,0.691555,0.983973,4.270475e-04,128.521851,8.643450e-04
82771002_2,9.500074,1.722189,2.002574,1.304077e-03,178.597244,52.291016,0.499375,0.008890,383.378815,0.139382,-515.577942,1.280869e-03,1.059804,1.851557,1.383258,0.979297,8.063276e-04,98.456902,5.788009e-17
82771002_3,8.564956,1.776963,2.003246,2.266835e-03,24.163748,49.513794,0.499204,0.004488,206.624451,0.065563,-557.685486,2.070052e-03,1.180172,1.830847,1.358215,0.978186,1.333789e-03,53.019714,4.367347e-04
82771002_4,9.368314,1.794851,1.996829,3.358016e-03,-60.085445,46.980427,0.499206,0.022453,93.918381,0.066347,-588.684082,3.144597e-03,1.179249,1.714181,1.061926,0.986710,8.919314e-04,21.473490,2.998317e-03
82771002_5,9.275356,1.562870,1.998725,1.301057e-14,-56.750549,46.783932,0.500335,0.007538,98.735710,0.179266,-586.073303,1.171623e-14,0.900817,1.741891,1.052604,0.992173,8.252288e-15,21.336618,8.013311e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44310191_96,8.667921,1.978963,1.979787,1.957909e-01,-127.956902,45.510365,0.505117,0.846437,114.081711,0.015084,-30.500875,1.837412e-01,1.389902,0.901214,1.435860,1.094604,2.559036e-02,499.661011,6.976674e-01
44310191_97,8.626774,2.025151,1.972213,1.092326e-01,-124.325623,45.366833,0.507058,0.858045,133.140076,0.000000,4.472719,1.037338e-01,1.366053,0.866658,1.447727,1.120325,3.276177e-04,534.880920,9.815037e-01
44310191_98,8.897718,1.988651,1.977096,1.221370e-01,-117.358086,47.030758,0.505808,0.831115,153.860657,0.029454,42.715897,1.151988e-01,1.429045,0.970187,1.478364,1.091279,1.844336e-02,572.944458,6.920288e-01


In [80]:
train_set = pd.read_csv('./tile_features/stamps_label.csv')
train_set = train_set.set_index('stamp_id')
train_set.rename(columns={'Unnamed: 0':'instance'},inplace=True)
train_set.loc[:,'stamp'] = [i.split('_')[0] for i in train_set.index]

In [81]:
train_set = train_set.join(df_template)

In [82]:
#train_set

In [83]:
stamps_data = '../data/stamp_locations.geo.json'
stamps = gpd.read_file(stamps_data)

In [84]:
# adding age to the feature
stamp_feature = stamps.loc[:,['id','age']]
stamp_feature.loc[:,'age'] = stamp_feature['age'].str.replace('Archean','Archaean')

In [85]:
train_set = pd.merge(left=train_set,right=stamp_feature,left_on='stamp',right_on='id',how='left')

In [86]:
train_set = train_set.drop(columns=['instance','comm','deps','id','stamp'])

In [87]:

# create dummy column for age and adding to dataset
train_set = train_set.join(pd.get_dummies(train_set["age"]))

In [88]:
dataset = train_set.copy()

In [89]:
# dropping age column
dataset.drop(columns=['age'], inplace=True)

In [90]:
dataset.rename(columns={'strat':'label'},inplace=True)

In [91]:
# drop inf rows
dataset = dataset[dataset['filtered_uranium_ppm']!=np.inf]
# drop nan
dataset = dataset.dropna()

In [92]:
#dataset

In [93]:
# # # factorizing labels 
# factor = pd.factorize(dataset['label'])
# dataset.loc[:,'label'] = factor[0]
# definitions = factor[1]
# print(dataset['label'].head())
# print(definitions)

In [94]:
dataset['label'].value_counts()

none     181685
Au         1297
Cu          990
other       316
Fe          277
PGE         206
Pb           29
Name: label, dtype: int64

In [95]:
# creating X matrix and y target vector
X = dataset.iloc[:,1:].values
y = dataset.iloc[:,0].values

print(X.shape)
print(y.shape)


(184800, 59)
(184800,)


In [96]:
le = LabelEncoder()
y = le.fit_transform(y)

In [97]:
le.classes_

array(['Au', 'Cu', 'Fe', 'PGE', 'Pb', 'none', 'other'], dtype=object)

In [23]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)

In [24]:
print(X_scaled.min())
print(X_scaled.max())

0.0
1.0000000000000002


In [30]:
# split train test
trainX, testX, trainY, testY = train_test_split(X_scaled, y,
                                                  test_size=0.25, random_state=42)

In [32]:
model = RandomForestClassifier()
# evaluate pipeline
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, trainX, trainY, scoring='accuracy', cv=3, n_jobs=-1)
print('Mean accuracy: %.3f' % mean(scores))

Mean accuracy: 0.983


In [33]:
# fit
model.fit(trainX, trainY)

# # Predicting the Test set results
predY = model.predict(testX)

In [36]:
testY = le.inverse_transform(testY)
predY = le.inverse_transform(predY)
# Making the Confusion Matrix
print(pd.crosstab(testY, predY, rownames=['Actual Classes'], colnames=['Predicted Classes']))

Predicted Classes  Au  Cu   none
Actual Classes                  
Au                  2   0    327
Cu                  1   2    265
Fe                  0   0     53
PGE                 0   0     59
Pb                  0   0      8
none                0   1  45399
other               0   0     83


In [37]:
# # class report

print(classification_report(testY,predY,le.classes_))

/Users/pedrojunqueira/anaconda3/envs/exploration/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          Au       0.67      0.01      0.01       329
          Cu       0.67      0.01      0.01       268
          Fe       0.00      0.00      0.00        53
         PGE       0.00      0.00      0.00        59
          Pb       0.00      0.00      0.00         8
        none       0.98      1.00      0.99     45400
       other       0.00      0.00      0.00        83

    accuracy                           0.98     46200
   macro avg       0.33      0.14      0.15     46200
weighted avg       0.97      0.98      0.97     46200



# Using Smote to oversample

In [40]:
# oversampling using SMOTE method
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [47]:
# Check if all classes are now with same sample
Counter(le.inverse_transform(y))

Counter({'none': 181685,
         'Cu': 181685,
         'other': 181685,
         'Au': 181685,
         'PGE': 181685,
         'Fe': 181685,
         'Pb': 181685})

In [48]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)

In [25]:
# split train test
trainX, testX, trainY, testY = train_test_split(X_scaled, y,
                                                  test_size=0.25, random_state=42)

In [57]:
model = RandomForestClassifier()
# evaluate pipeline
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, trainX, trainY, scoring='accuracy', cv=3, n_jobs=-1)
print('Mean accuracy: %.3f' % mean(scores))

Mean accuracy: 0.994


In [58]:
# fit
model.fit(trainX, trainY)

# # Predicting the Test set results
predY = model.predict(testX)

In [59]:
testY = le.inverse_transform(testY)
predY = le.inverse_transform(predY)
# Making the Confusion Matrix
print(pd.crosstab(testY, predY, rownames=['Actual Classes'], colnames=['Predicted Classes']))

Predicted Classes     Au     Cu     Fe    PGE     Pb   none  other
Actual Classes                                                    
Au                 45224      3      3      1      0     62      0
Cu                    22  45457      0      0      0     37      0
Fe                     0      0  45527      0      0      2      0
PGE                    0      0      0  45158      0      5      0
Pb                     0      0      0      0  45588      1      0
none                 522    305    108     32      5  44249     90
other                  0      0      0      0      0      8  45540


In [60]:
# # class report

print(classification_report(testY,predY,le.classes_))

              precision    recall  f1-score   support

          Au       0.99      1.00      0.99     45293
          Cu       0.99      1.00      1.00     45516
          Fe       1.00      1.00      1.00     45529
         PGE       1.00      1.00      1.00     45163
          Pb       1.00      1.00      1.00     45589
        none       1.00      0.98      0.99     45311
       other       1.00      1.00      1.00     45548

    accuracy                           1.00    317949
   macro avg       1.00      1.00      1.00    317949
weighted avg       1.00      1.00      1.00    317949



# Oversampling only Validation Train set and predicting on an inbalanced test set

In [61]:
# creating X matrix and y target vector
X = dataset.iloc[:,1:].values
y = dataset.iloc[:,0].values

print(X.shape)
print(y.shape)

(184800, 59)
(184800,)


In [62]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)

In [63]:
le = LabelEncoder()
y = le.fit_transform(y)

In [67]:
# split train test
trainX, testX, trainY, testY = train_test_split(X_scaled, y,
                                                  test_size=0.25, random_state=42)

In [68]:
# oversample only train set

print(trainX.shape)

# oversampling using SMOTE method
oversample = SMOTE()
trainX, trainY = oversample.fit_resample(trainX, trainY)

# Check if all classes are now with same sample
Counter(le.inverse_transform(trainY))

(138600, 59)


Counter({'none': 136285,
         'Au': 136285,
         'Cu': 136285,
         'Fe': 136285,
         'other': 136285,
         'PGE': 136285,
         'Pb': 136285})

In [69]:
# validation test still inbalanced

Counter(le.inverse_transform(testY))

Counter({'none': 45400,
         'Cu': 268,
         'Au': 329,
         'other': 83,
         'PGE': 59,
         'Pb': 8,
         'Fe': 53})

In [70]:
model = RandomForestClassifier()
# evaluate pipeline
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, trainX, trainY, scoring='accuracy', cv=3, n_jobs=-1)
print('Mean accuracy: %.3f' % mean(scores))

Mean accuracy: 0.997


In [71]:
# fit
model.fit(trainX, trainY)

# # Predicting the Test set results
predY = model.predict(testX)

In [72]:
testY = le.inverse_transform(testY)
predY = le.inverse_transform(predY)
# Making the Confusion Matrix
print(pd.crosstab(testY, predY, rownames=['Actual Classes'], colnames=['Predicted Classes']))

Predicted Classes   Au   Cu  Fe  PGE  Pb   none  other
Actual Classes                                        
Au                  82    4   1    1   0    241      0
Cu                   5   44   0    2   0    216      1
Fe                   1    0   7    0   0     45      0
PGE                  1    2   0    9   0     47      0
Pb                   0    0   0    0   0      8      0
none               278  172  95   16   7  44772     60
other                0    2   0    0   0     75      6


In [73]:
# # class report

print(classification_report(testY,predY,le.classes_))

              precision    recall  f1-score   support

          Au       0.22      0.25      0.24       329
          Cu       0.20      0.16      0.18       268
          Fe       0.07      0.13      0.09        53
         PGE       0.32      0.15      0.21        59
          Pb       0.00      0.00      0.00         8
        none       0.99      0.99      0.99     45400
       other       0.09      0.07      0.08        83

    accuracy                           0.97     46200
   macro avg       0.27      0.25      0.25     46200
weighted avg       0.97      0.97      0.97     46200



In [77]:
# save model to disk


joblib.dump(model, 'RF_imbalanced_trained.pkl') 


['RF_imbalanced_trained.pkl']

In [27]:
# load model from disk
model_RF_loaded = joblib.load('RF_imbalanced_trained.pkl')

In [28]:
model_RF_loaded.predict(testX)


array([5, 5, 5, ..., 5, 5, 5])

In [29]:
Counter(le.inverse_transform(model_RF_loaded.predict(testX)))

Counter({'none': 45404,
         'Fe': 103,
         'Au': 367,
         'Cu': 224,
         'PGE': 28,
         'other': 67,
         'Pb': 7})

# Pre-Process Test Set

In [2]:
path = '/Users/pedrojunqueira/Desktop/explore_australia/MyNBs/tile_features'

In [8]:
test_feature_paths = [path for path in os.listdir(path) if 'test' in path]

In [9]:
test_feature_paths[0]

'total_magnetic_intensity_test.csv'

In [62]:
df_template_t = pd.read_csv('./tile_features/total_magnetic_intensity_test.csv')
df_template_t = df_template_t.set_index('tile_id')
df_template_t = df_template_t.drop(columns='mean_')
df_template_t

for file in test_feature_paths:
    df = pd.read_csv(f'./tile_features/{file}')
    col_name = file.split('.')[:-1][0]
    col_name = '_'.join(col_name.split('_')[:-1])
    df.rename(columns={'mean_':col_name},inplace=True)
    df = df.set_index('tile_id')
    df_template_t = df_template_t.join(df)


In [63]:
# order column as in the train set
df_template_t = df_template_t.loc[:,['filtered_thorium_ppm', 'aloh_groun_content',
       'thermal_infrared_gypsum_index', 'mgoh_group_content',
       'variable_reduction_to_pole', 'filtered_terrestrial_dose',
       'tir_quartz_index', 'ferrous_iron_index', 'total_magnetic_intensity',
       'opaque_index', 'bouger_gravity_anomaly', 'kaolin_group_index',
       'ferric_oxide_content', 'filtered_uranium_ppm',
       'filtered_potassium_pct', 'thermal_infrared_silica_index',
       'ferrous_iron_content', 'isostatic_residual_gravity_anomaly',
       'aloh_group_composition']]

In [68]:
df_template_t.loc[:,'stamp'] = [i.split('_')[0] for i in df_template_t.index]
df_template_t = df_template_t.reset_index()

In [69]:
test_set = pd.merge(left=df_template_t,right=stamp_feature,left_on='stamp',right_on='id',how='left')
test_set = test_set.drop(columns=['id','stamp'])

In [71]:
# create dummy column for age and adding to dataset
test_set = test_set.join(pd.get_dummies(test_set["age"]))


In [72]:
test_dataset = test_set.copy()
# dropping age column
test_dataset.drop(columns=['age'], inplace=True)


In [73]:
# drop inf rows
test_dataset = test_dataset[test_dataset['filtered_uranium_ppm']!=np.inf]
# drop nan
test_dataset = test_dataset.dropna()

In [74]:
test_dataset

,tile_id,filtered_thorium_ppm,aloh_groun_content,thermal_infrared_gypsum_index,mgoh_group_content,variable_reduction_to_pole,filtered_terrestrial_dose,tir_quartz_index,ferrous_iron_index,total_magnetic_intensity,...,Palaeozoic,Paleoproterozoic,Permian,Precambrian,Proterozoic,Quaternary,Silurian,Tertiary,Triassic,unknown
0,27801422_1,14.583928,1.841413,1.998361,0.006573,-23.938871,68.840126,0.500420,0.319090,-36.350555,...,0,0,0,0,0,0,0,0,0,0
1,27801422_2,14.019094,1.834027,1.996632,0.006522,-22.846136,64.751541,0.500852,0.470932,-38.031258,...,0,0,0,0,0,0,0,0,0,0
2,27801422_3,15.167210,1.845156,1.995486,0.017611,-22.193148,72.486549,0.501140,0.480541,-38.811146,...,0,0,0,0,0,0,0,0,0,0
3,27801422_4,14.073159,1.830946,1.995834,0.033020,-27.158535,65.638809,0.501054,0.536202,-43.594757,...,0,0,0,0,0,0,0,0,0,0
4,27801422_5,11.649189,1.835220,1.997195,0.027035,-29.215328,49.744656,0.500713,0.537161,-45.382557,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62195,33920526_96,5.635985,1.958360,1.989640,0.052890,-117.751373,19.386864,0.502622,0.692879,13.258567,...,0,0,0,0,0,0,0,0,0,0
62196,33920526_97,15.240803,1.939375,1.992917,0.028339,191.223267,54.273273,0.501793,0.661870,46.915806,...,0,0,0,0,0,0,0,0,0,0
62197,33920526_98,8.546436,1.925508,1.995121,0.026268,988.674805,29.854349,0.501234,0.681896,726.817505,...,0,0,0,0,0,0,0,0,0,0
62198,33920526_99,9.816708,1.954946,1.991299,0.042597,850.099609,31.823103,0.502207,0.599986,658.049500,...,0,0,0,0,0,0,0,0,0,0


In [102]:
dscols = dataset.columns[1:]
tdscols = test_dataset.columns

In [103]:
# missing columns in testdataset
misscols = [col for col in dscols if col not in tdscols]

In [104]:
misscols

['Cambrian',
 'Early Carboniferous',
 'Early Ordovician',
 'Holocene',
 'Jurassic',
 'Paleogene',
 'Pleistocene',
 'Pliocene']

In [105]:
extracol = [col for col in tdscols if col not in dscols]

In [106]:
extracol

['tile_id', 'Middle Triassic']

In [107]:
test_dataset.drop(columns='Middle Triassic', inplace=True)
test_dataset.shape

(61700, 52)

In [109]:
# add missing columns
for col in misscols:
    test_dataset.loc[:,col] = 0
test_dataset.shape    

(61700, 60)

In [116]:
# same columns as in the train same order
test_dataset = test_dataset.loc[:,['tile_id', 'filtered_thorium_ppm', 'aloh_groun_content',
       'thermal_infrared_gypsum_index', 'mgoh_group_content',
       'variable_reduction_to_pole', 'filtered_terrestrial_dose',
       'tir_quartz_index', 'ferrous_iron_index', 'total_magnetic_intensity',
       'opaque_index', 'bouger_gravity_anomaly', 'kaolin_group_index',
       'ferric_oxide_content', 'filtered_uranium_ppm',
       'filtered_potassium_pct', 'thermal_infrared_silica_index',
       'ferrous_iron_content', 'isostatic_residual_gravity_anomaly',
       'aloh_group_composition', 'Archaean', 'Cainozoic', 'Cambrian',
       'Carboniferous', 'Cretaceous', 'Devonian', 'Early Carboniferous',
       'Early Cretaceous', 'Early Devonian', 'Early Ordovician',
       'Early Palaeozoic', 'Early Permian', 'Early Pliocene', 'Holocene',
       'Jurassic', 'Late Cambrian', 'Late Carboniferous', 'Late Ordovician',
       'Late Palaeozoic', 'Late Permian', 'Late Silurian', 'Late Triassic',
       'Mesoproterozoic', 'Middle Devonian', 'Neoproterozoic', 'Ordovician',
       'Palaeoproterozoic', 'Palaeozoic', 'Paleogene', 'Paleoproterozoic',
       'Permian', 'Pleistocene', 'Pliocene', 'Precambrian', 'Proterozoic',
       'Quaternary', 'Silurian', 'Tertiary', 'Triassic', 'unknown']]

In [117]:
# creating X matrix and y target vector
test_X = test_dataset.iloc[:,1:].values
tile_id = test_dataset.iloc[:,0].values

In [118]:
test_X.shape

(61700, 59)

In [120]:
min_max_scaler = preprocessing.MinMaxScaler()
test_X = min_max_scaler.fit_transform(test_X)

# Predict

In [124]:
# load model from disk
model_RF_loaded = joblib.load('RF_imbalanced_trained.pkl')

In [125]:
model_RF_loaded.predict(test_X)

array([5, 5, 5, ..., 5, 5, 5])

In [127]:
Counter(le.inverse_transform(model_RF_loaded.predict(test_X)))

Counter({'none': 61700})

In [ ]:
# def get_models():
#     models, names = list(), list()
#     # LDA
#     models.append(LinearDiscriminantAnalysis())
#     names.append('LDA')
#     # SVM
#     models.append(LinearSVC())
#     names.append('SVM')
#     # Bagging
#     models.append(BaggingClassifier(n_estimators=1000))
#     names.append('BAG')
#     # RF
#     models.append(RandomForestClassifier(n_estimators=1000))
#     names.append('RF')
#     # ET
#     models.append(ExtraTreesClassifier(n_estimators=1000))
#     names.append('ET')
#     return models, names


# def evaluate_model(X, y, model):
#     # define evaluation procedure
#     cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
#     # evaluate model
#     scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#     return scores

# models, names = get_models()
# results = list()
# # evaluate each model
# for i in range(len(models)):
#     # evaluate the model and store results
#     scores = evaluate_model(trainX, trainY, models[i])
#     results.append(scores)
#     # summarize performance
#     print('>%s %.3f (%.3f)' % (names[i], mean(scores), std(scores)))
# # plot the results
# plt.boxplot(results, labels=names, showmeans=True)
# plt.show()


# # # Fitting Random Forest Classification to the Training set
# classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
# classifier.fit(trainX, trainY)